In [1]:
from haystack.components.fetchers.link_content import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.rankers import TransformersSimilarityRanker
from haystack.components.generators import GPTGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack import Pipeline
from haystack.components.retrievers import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.dataclasses import Document
from haystack.components.writers import DocumentWriter

from dotenv import load_dotenv
import os

load_dotenv("../../.env")

openai_api_key = os.getenv("OPENAI_API_KEY")


/Users/macpro/anaconda3/envs/llm-bjj-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
with open("urls.txt", "r") as f:
    urls = f.readlines()

clean_urls = [url.strip() for url in urls]

In [3]:
document_store = InMemoryDocumentStore()
fetcher = LinkContentFetcher()
converter = HTMLToDocument()
splitter = DocumentSplitter(split_length=100, split_overlap=5)
writer = DocumentWriter(document_store = document_store)
retriever = InMemoryBM25Retriever(document_store = document_store)
ranker = TransformersSimilarityRanker()
ranker.warm_up()
llm =  GPTGenerator(api_key = openai_api_key, model_name = "gpt-4")

prompt_template = """
According to these documents:

{% for doc in documents %}
  {{ doc.content }}
{% endfor %}

Answer the given question: {{question}}
Answer:
"""


prompt_builder = PromptBuilder(template=prompt_template)

## Indexing pipeline

In [4]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_component(instance=fetcher, name="fetcher")
indexing_pipeline.add_component(instance=converter, name="converter")
indexing_pipeline.add_component( instance=splitter, name="splitter")
indexing_pipeline.add_component(instance=writer, name="writer")


indexing_pipeline.connect("fetcher.streams", "converter.sources")
indexing_pipeline.connect("converter.documents", "splitter.documents")
indexing_pipeline.connect("splitter.documents", "writer.documents")

In [5]:
indexing_pipeline.run(data={"fetcher": {"urls": clean_urls}})

{'writer': {'documents_written': 176}}

### RAG Pipeline

In [6]:
basic_rag_pipeline = Pipeline()

basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", llm)


basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

basic_rag_pipeline.draw("../../images/mechanical_models_rag.png")

In [7]:
question = "Design a complete dominance game plan for a tall person"

response = basic_rag_pipeline.run({"retriever": {"query": question}, 
                                   "prompt_builder": {"question": question}})


Ranking by BM25...: 100%|██████████| 176/176 [00:00<00:00, 27803.00 docs/s]


In [ ]:
response['llm']['replies'][0].split("\n")

In [8]:
question = "Design a sequence of BJJ movements for a blue belt who is\
            female, 192 lbs, and 5'10 tall. Your sequence should apply the mental models presented to you\
            and should enable the competitor to attack and succeed in their attacks,and be able\
            to successfully defend against opponents. Your answer should suggest \
            multiple strategies for dealing with opponents of various weights and heights\
            as well as opponents of the opposite gender."

response = basic_rag_pipeline.run({"retriever": {"query": question}, 
                                   "prompt_builder": {"question": question}})


Ranking by BM25...: 100%|██████████| 176/176 [00:00<00:00, 28066.21 docs/s]


In [11]:
response['llm']['replies'][0].split("\n")

["Here is a hypothetical sequence of Brazilian Jiu-Jitsu movements developed with the consideration of the mental models presented and drawn upon the general knowledge of BJJ. This sequence is tailored to enhance a tall, strong female competitor's performance, and provide her strategies against opponents of diverse sizes and genders:",
 '',
 '1. Effective Breathing: Our competitor should focus on her breathing to avoid signs of panic and as an effective way to regulate her energy consumption.',
 '',
 '2. Stance: The competitor should establish her base with a staggered stance (since taller practitioners often favor it). This stance offers the advantages of both stability and mobility, which are crucial against smaller or heavier opponents. ',
 '',
 "3. Inside Channel Control: She should then seek to control the “inside channel”. Because she's taller than most opponents, she can leverage her longer limbs to dominate this area. Inside channel control will allow her to manipulate opponent

In [19]:
question = "explain the theory of alignment in BJJ with the documents provided\
            and provide a sequence of movements that demonstrate the theory of alignment\
            in BJJ"

response = basic_rag_pipeline.run({"retriever": {"query": question}, 
                                   "prompt_builder": {"question": question}})


Ranking by BM25...: 100%|██████████| 176/176 [00:00<00:00, 22116.29 docs/s]


In [20]:
response['llm']['replies'][0].split("\n")

['The theory of alignment in Brazilian Jiu-Jitsu (BJJ), innovated by Rob Biernacki of Island Top Team, is a framework that describes the mechanical aspects of the martial art. It comprises of three components: Posture, Structure, and Base. ',
 '',
 '1. Posture - This refers to the effective positioning of essential body parts such as the neck, core, and spine. How one maintains their posture during a BJJ match can significantly impact their performance.',
 '2. Structure - The concept of structure in the theory of alignment pertains to the efficient usage of your limbs.',
 '3. Base - The base component alludes to your ability to generate and absorb force relative to your goals set for the match.',
 '',
 "These elements have the potential to multiply your force, thus allowing you to create openings and better attack your opponent’s body. Constant evaluation of every technique is important, considering how well it preserves your alignment and how well it breaks your opponent's alignment."

In [23]:
question = "what is ratchet control?"

response = basic_rag_pipeline.run({"retriever": {"query": question}, 
                                   "prompt_builder": {"question": question}})

response['llm']['replies'][0].split("\n")


Ranking by BM25...: 100%|██████████| 176/176 [00:00<00:00, 34215.41 docs/s]


["Ratchet control is a technique used when controlling a limb or the head, where rotation is added to increase effectiveness and leverage. This is typically more powerful if the lever is kept at a perpendicular angle. It can be either internal or external, based on the direction of rotation. Internal ratchet control rotates a limb against its natural range of motion, such as in Kimuras and heel hooks. External ratchet control rotates a limb into its natural range of motion. This technique is particularly effective because it attacks the opponent's posture and/or skeletal structure, reducing their ability to rely on their muscles. It is one of the most effective ways to complete a submission when an opponent is using strength to defend."]